# Schedule optimization

### Model overview

The following model creates an optimized schedule for a set of workers. The schedule is for <span style="color:blue">one week</span> and consists of <span style="color:blue">two shifts per day</span>.

**Optimization**: The schedule aims to produce the <span style="color:green">_lowest cost staffing arrangement_</span>

**Variables**:
* Every worker/shift combination
* Total cost accounts for overtime and regular time

**Constraints**:
* Total number of workers staffed equals total number of workers required for each shift
* Workers must stay between global minimum and maximum number of shifts
* Workers can only be scheduled when they are available (handled in decision variable 'x')
* At least one manager staffed per shift

**Output**: 
* Total cost of proposed schedule
* Dashboard with full schedule
* Names of nurses assigned to each shift

### Data import

In [1]:
# Import required packages
import pandas as pd
import numpy as np
from gurobipy import *

Create a list of shifts and workers

In [2]:
shiftList = ['Monday1','Monday2','Tuesday1','Tuesday2','Wednesday1','Wednesday2'
             ,'Thursday1','Thursday2','Friday1','Friday2','Saturday1','Saturday2','Sunday1','Sunday2']
workerList = ['EE01','EE02','EE03','EE04','EE05','EE06','EE07','EE08','EE09','EE10']

Define how many workers are needed for each shift

In [3]:
shiftReq = [3,2,4,4,5,4,5,4,2,4,5,4,3,5]
shiftRequirements  = { s : shiftReq[i] for i,s in enumerate(shiftList) }

Clarify availability of each worker for each shift

In [4]:
# Assume everyone is available
availability = pd.DataFrame(np.ones((len(workerList), len(shiftList))), index=workerList, columns=shiftList)

# For illustration, assume following people are unavailable: EE02 on Tuesday1, EE05 on Saturday2, EE08 on Thursday1
availability.at['EE02','Tuesday1'] = 0
availability.at['EE05','Saturday2'] = 0
availability.at['EE08','Thursday1'] = 0

# Create availability dictionary to be used in decision variable bounding
avail = {(w,s) : availability.loc[w,s] for w in workerList for s in shiftList}

Specify who is a manager

In [5]:
mgmtList = ['EE01','EE03','EE05','EE07']
nonmgmtList = [x for x in workerList if x not in mgmtList]

Define total shift cost per worker

In [6]:
# Cost of a regular shift
regCost = [200,100,225,110,190,105,210,120,95,100]

# Cost of overtime shift
OTCost = [1.5*x for x in regCost]

regularCost  = { w : regCost[i] for i,w in enumerate(workerList) }
overtimeCost  = { w : OTCost[i] for i,w in enumerate(workerList) }

### Input assumptions

In [7]:
# Range of shifts that every workers is required to stay between
minShifts = 3
maxShifts = 7

# Number of shifts to trigger overtime
OTTrigger = 5

### Create model

In [8]:
model = Model("Workers Scheduling")

### Define decision variables

In [9]:
# ub ensures that workers are only staffed when they are available
x = model.addVars(workerList, shiftList, ub=avail, vtype=GRB.BINARY, name='x')

In [10]:
regHours = model.addVars(workerList, name='regHrs')
overtimeHours = model.addVars(workerList, name='overtimeHrs')
overtimeTrigger = model.addVars(workerList, name = "OT_Trigger", vtype = GRB.BINARY)

### Add constraints

In [11]:
# Ensure proper number of workers are scheduled

shiftReq = model.addConstrs((
    (x.sum('*',s) == shiftRequirements[s] for s in shiftList)
), name='shiftRequirement')

In [12]:
# Differentiate between regular time and overtime

## Decompose total shifts for each worker into regular shifts and OT shifts
regOT1 = model.addConstrs((regHours[w] + overtimeHours[w] == x.sum(w,'*') for w in workerList))
## Ensure that regular shifts are accounted for first for each nurse before counting OT shifts
regOT2 = model.addConstrs((regHours[w] <= OTTrigger for w in workerList))
## Only allow the OT trigger to come on when regular shift count is greater than regular shift limit
regOT3 = model.addConstrs((regHours[w] / OTTrigger >= overtimeTrigger[w] for w in workerList))

In [13]:
# Ensure each worker stays within min and max shift bounds

minShiftsConstr = model.addConstrs(((
    x.sum(w,'*') >= minShifts for w in workerList)
), name='minShifts')

maxShiftsConstr = model.addConstrs(((
    x.sum(w,'*') <= maxShifts for w in workerList)
), name='maxShifts')

In [14]:
# Ensure every shift has at least one manager

for s in shiftList:
    model.addConstr((quicksum(x.sum(m,s) for m in mgmtList) >= 1), name='mgmtStaffing'+str(s))

### Objective function

In [15]:
# Minimize total cost, accounting for pay difference between regular time and overtime

model.ModelSense = GRB.MINIMIZE

Cost = 0
Cost += (quicksum(regularCost[w]*regHours[w] + overtimeCost[w]*overtimeHours[w] for w in workerList))

model.setObjective(Cost)

### Run optimization

In [16]:
model.write("Optimized_Scheduling.lp")
file = open("Optimized_Scheduling.lp", 'r')
print(file.read())
file.close()

\ Model Workers Scheduling
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  200 regHrs[EE01] + 100 regHrs[EE02] + 225 regHrs[EE03] + 110 regHrs[EE04]
   + 190 regHrs[EE05] + 105 regHrs[EE06] + 210 regHrs[EE07]
   + 120 regHrs[EE08] + 95 regHrs[EE09] + 100 regHrs[EE10]
   + 300 overtimeHrs[EE01] + 150 overtimeHrs[EE02]
   + 337.5 overtimeHrs[EE03] + 165 overtimeHrs[EE04]
   + 285 overtimeHrs[EE05] + 157.5 overtimeHrs[EE06]
   + 315 overtimeHrs[EE07] + 180 overtimeHrs[EE08]
   + 142.5 overtimeHrs[EE09] + 150 overtimeHrs[EE10]
Subject To
 shiftRequirement[Monday1]: x[EE01,Monday1] + x[EE02,Monday1]
   + x[EE03,Monday1] + x[EE04,Monday1] + x[EE05,Monday1] + x[EE06,Monday1]
   + x[EE07,Monday1] + x[EE08,Monday1] + x[EE09,Monday1] + x[EE10,Monday1]
   = 3
 shiftRequirement[Monday2]: x[EE01,Monday2] + x[EE02,Monday2]
   + x[EE03,Monday2] + x[EE04,Monday2] + x[EE05,Monday2] + x[EE06,Monday2]
   + x[EE07,Monday2] + x[EE08,Monday2] + x[EE09,Monday2] + x[E

In [17]:
model.optimize()

Optimize a model with 78 rows, 170 columns and 666 nonzeros
Variable types: 20 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 12660.000000
Presolve removed 20 rows and 13 columns
Presolve time: 0.00s
Presolved: 58 rows, 157 columns, 623 nonzeros
Variable types: 20 continuous, 137 integer (137 binary)

Root relaxation: objective 7.535000e+03, 106 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7535.0000000 7535.00000  0.00%     -    0s

Explored 0 nodes (106 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 7535 12660 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.535000000000e+0

### Output optimized schedule

In [18]:
print('Total cost = $' + str(model.ObjVal))

Total cost = $7535.0


In [19]:
sol = pd.DataFrame(data={'Solution':model.X}, index=model.VarName)
sol = sol.iloc[0:len(x)]

dashboard = pd.DataFrame(index = workerList, columns = shiftList)
for w in workerList:
    for s in shiftList:
        dashboard.at[w,s] = sol.loc['x['+w+','+s+']',][0]
        
dashboard

,Monday1,Monday2,Tuesday1,Tuesday2,Wednesday1,Wednesday2,Thursday1,Thursday2,Friday1,Friday2,Saturday1,Saturday2,Sunday1,Sunday2
EE01,-0,-0,-0,1,-0,-0,-0,1,-0,1,0,-0,-0,-0
EE02,-0,-0,0,1,1,0,1,-0,-0,1,1,0,1,1
EE03,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,0,1,1,1
EE04,1,0,-0,-0,0,1,0,-0,-0,1,1,1,1,1
EE05,1,1,-0,-0,1,-0,0,-0,1,-0,1,0,-0,-0
EE06,-0,1,1,-0,1,-0,1,1,-0,0,1,1,-0,0
EE07,0,-0,1,-0,-0,1,1,-0,-0,-0,-0,-0,-0,-0
EE08,0,-0,1,-0,-0,1,0,1,1,-0,-0,-0,-0,1
EE09,-0,-0,1,1,1,-0,1,0,-0,1,0,1,0,1
EE10,1,-0,0,1,1,1,1,1,0,-0,1,-0,-0,-0


In [20]:
shiftAssignments = {}
for s in shiftList:
    shiftAssignments.update({s: list(dashboard[dashboard[s] == 1].loc[:,].index)})
    
shiftAssignments

{'Monday1': ['EE04', 'EE05', 'EE10'],
 'Monday2': ['EE05', 'EE06'],
 'Tuesday1': ['EE06', 'EE07', 'EE08', 'EE09'],
 'Tuesday2': ['EE01', 'EE02', 'EE09', 'EE10'],
 'Wednesday1': ['EE02', 'EE05', 'EE06', 'EE09', 'EE10'],
 'Wednesday2': ['EE04', 'EE07', 'EE08', 'EE10'],
 'Thursday1': ['EE02', 'EE06', 'EE07', 'EE09', 'EE10'],
 'Thursday2': ['EE01', 'EE06', 'EE08', 'EE10'],
 'Friday1': ['EE05', 'EE08'],
 'Friday2': ['EE01', 'EE02', 'EE04', 'EE09'],
 'Saturday1': ['EE02', 'EE04', 'EE05', 'EE06', 'EE10'],
 'Saturday2': ['EE03', 'EE04', 'EE06', 'EE09'],
 'Sunday1': ['EE02', 'EE03', 'EE04'],
 'Sunday2': ['EE02', 'EE03', 'EE04', 'EE08', 'EE09']}